# 第四章 句子滑窗检索

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

读取数据库

In [ ]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["data/人工智能.pdf"]
).load_data()

In [ ]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

这里通过将 documents 中各个文档的文本连接成一个字符串，然后创建了一个 Document 实例，该实例代表了整个文档集合。

In [ ]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [ ]:
# 将中文标点符号替换成英文标点符号，方便后续处理
# 如果是英文文档，可以跳过这一步
# 不处理的话，会导致无法正确切分中文句子，会影响后续sentence_window的大小，导致输入长度大于gpt-3.5-turbo的最大限制
document.text=document.text.replace('。','. ')
document.text=document.text.replace('！','! ')
document.text=document.text.replace('？','? ')

## 一、句子滑窗检索设置

创建了一个名为 node_parser 的解析器对象，指定了窗口大小为3，原始文本元数据键被设置为``original_text``。这样创建的解析器可以用于从文本中提取节点

In [ ]:
from llama_index.node_parser import SentenceWindowNodeParser

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

定义一个中文文本字符串  
使用 node_parser 的 get_nodes_from_documents 方法从提供的文本中提取节点。

In [ ]:
text = "你好. 你怎么样? 我很好!  "

nodes = node_parser.get_nodes_from_documents([Document(text=text)])

每个单独的词

In [ ]:
print([x.text for x in nodes])

原整句

In [ ]:
print(nodes[1].metadata["window"])

In [ ]:
text = "你好. 吧台. 猫狗. 老鼠"

nodes = node_parser.get_nodes_from_documents([Document(text=text)])

In [ ]:
print([x.text for x in nodes])

In [ ]:
print(nodes[0].metadata["window"])

### 2.1 创建索引

使用 `OpenAI` 的 `GPT-3.5-turbo` 模型创建了一个语言模型的实例，设置了温度参数为0.1。

In [ ]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

使用 `ServiceContext.from_defaults` 方法创建了一个 `ServiceContext` 对象，该对象包含了用于索引构建的服务相关的上下文信息，包括语言模型、嵌入模型以及节点解析器。

In [ ]:
from llama_index import ServiceContext

sentence_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-zh-v1.5",
    node_parser=node_parser,
)

使用 `VectorStoreIndex.from_documents` 方法创建了一个 `VectorStoreIndex` 对象，该对象用于存储和检索与文档相关的向量信息。

In [ ]:
from llama_index import VectorStoreIndex

sentence_index = VectorStoreIndex.from_documents(
    [document], service_context=sentence_context
)

将创建的索引持久化到指定目录`（"./sentence_index"）`。这样做可以在之后的运行中重新加载索引，而不必重新构建。

In [ ]:
sentence_index.storage_context.persist(persist_dir="./sentence_index")

检查索引文件是否存在，如果不存在则重新构建,如果存在，它将使用 `load_index_from_storage` 方法从已有的索引文件中加载索引，而不是重新构建。

In [ ]:
# This block of code is optional to check
# if an index file exist, then it will load it
# if not, it will rebuild it

import os
from llama_index import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index import load_index_from_storage

if not os.path.exists("./sentence_index"):
    sentence_index = VectorStoreIndex.from_documents(
        [document], service_context=sentence_context
    )

    sentence_index.storage_context.persist(persist_dir="./sentence_index")
else:
    sentence_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./sentence_index"),
        service_context=sentence_context
    )

### 2.2 创建后处理

使用 `MetadataReplacementPostProcessor` 类创建了一个后处理器实例，设置了目标元数据键为 `window`。该后处理器的作用是替换目标元数据键的内容。

In [ ]:
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor

postproc = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)

使用 `NodeWithScore` 类，将原始节点列表中的每个节点与一个分数关联，形成带分数的节点列表。  
使用 `deepcopy` 函数创建了原始节点列表的深度拷贝，以便后续比较。

In [ ]:
from llama_index.schema import NodeWithScore
from copy import deepcopy

scored_nodes = [NodeWithScore(node=x, score=1.0) for x in nodes]
nodes_old = [deepcopy(n) for n in nodes]

In [ ]:
nodes_old[1].text

使用后处理器的 `postprocess_nodes` 方法，替换了带分数的节点列表中目标元数据键的内容。

In [ ]:
replaced_nodes = postproc.postprocess_nodes(scored_nodes)

In [ ]:
print(replaced_nodes[1].text)

### 2.3 增设重新排序块

使用 `SentenceTransformerRerank` 类创建了一个后处理器实例，设置了参数 `top_n` 为 2，以及使用的模型为 "BAAI/bge-reranker-base"。

In [ ]:
from llama_index.indices.postprocessor import SentenceTransformerRerank

# BAAI/bge-reranker-base
# link: https://huggingface.co/BAAI/bge-reranker-base
rerank = SentenceTransformerRerank(
    top_n=2, model="BAAI/bge-reranker-base"
)

创建了一个包含查询文本的 `QueryBundle` 对象，该查询文本为 "我想要只狗."。  
创建了一个包含两个带分数的节点的列表，这些节点分别表示包含 "这是只猫" 和 "这是只狗" 文本的文本节点，分数分别为 0.6 和 0.4。

In [ ]:
from llama_index import QueryBundle
from llama_index.schema import TextNode, NodeWithScore

query = QueryBundle("我想要只狗.")

scored_nodes = [
    NodeWithScore(node=TextNode(text="这是只猫"), score=0.6),
    NodeWithScore(node=TextNode(text="这是只狗"), score=0.4),
]

使用 `SentenceTransformerRerank` 类的 `postprocess_nodes` 方法，对带分数的节点列表进行重新排名，考虑到查询文本。重新排名的节点将基于预训练的句子转换模型。

In [ ]:
reranked_nodes = rerank.postprocess_nodes(
    scored_nodes, query_bundle=query
)

输出了重新排名后的节点列表中的文本和分数。这里展示了句子转换模型对节点重新排名的效果。

In [ ]:
print([(x.text, x.score) for x in reranked_nodes])

'''
[('This is a dog', 0.91827345), ('This is a cat', 0.0014040739)]
'''

### 2.4 运行索引引擎

使用 `as_query_engine` 方法将 `sentence_index` 转换为查询引擎对象 `sentence_window_engine`。  
在这里，设置了相似性`（similarity）`的 `top k` 为 6，并传入了 `node_postprocessors` 参数，其中包含了之前创建的 `postproc` 和 `rerank` 后处理器。

In [ ]:
sentence_window_engine = sentence_index.as_query_engine(
    similarity_top_k=6, node_postprocessors=[postproc, rerank]
)

使用查询引擎的 `query` 方法执行了一个查询，查询的内容是 "在人工智能领域建功立业的关键是什么?"。查询引擎将使用之前设置的后处理器进行节点后处理。

In [ ]:
window_response = sentence_window_engine.query(
    "在人工智能领域建功立业的关键是什么?"
)

使用 `LLAMA` 框架提供的 `display_response` 函数展示了查询的响应结果。这通常包括与查询匹配的一组节点，以及它们的文本、分数等信息。  
这种方式可以在`Notebook`环境中更好地可视化和理解查询的结果。

In [ ]:
from llama_index.response.notebook_utils import display_response

display_response(window_response)

## 二、合并上述操作

`documents`: 要构建索引的文档列表。  
`llm`: OpenAI 语言模型实例。  
`embed_model`: 嵌入模型的名称或路径。  
`sentence_window_size`: 句子窗口的大小。  
`save_dir`: 持久化索引的目录。  
  
创建一个句子窗口的节点解析器（node_parser）。  
创建一个包含语言模型和节点解析器等上下文信息的 ServiceContext。  
如果指定的目录中不存在索引，则创建一个基于提供的文档的 VectorStoreIndex 并将其持久化到指定目录。  
如果目录中已存在索引文件，则从文件中加载索引。  
返回构建的句子窗口索引。

In [ ]:
import os
from llama_index import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import load_index_from_storage


def build_sentence_window_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-zh-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index",
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents, service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index


`sentence_index`: 已构建的句子窗口索引。  
`similarity_top_k`: 相似性查询的 top k。  
`rerank_top_n`: 重新排名的 top n。  
  
定义了两个后处理器：`postproc` 用于替换元数据键，`rerank` 用于使用句子转换模型重新排名节点。  
创建一个查询引擎 `sentence_window_engine`，将句子窗口索引转换为查询引擎，并使用定义的后处理器。  
返回构建的查询引擎。

In [ ]:
def get_sentence_window_query_engine(
    sentence_index, similarity_top_k=6, rerank_top_n=2
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

调用之前定义的 `build_sentence_window_index` 函数，传入文档列表、语言模型实例和保存目录，以构建句子窗口索引。

In [ ]:
from llama_index.llms import OpenAI

index = build_sentence_window_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    save_dir="./sentence_index",
)


调用之前定义的 `get_sentence_window_query_engine` 函数，传入构建的句子窗口索引和相似性 `top k`，以获取句子窗口的查询引擎。  
在这里，`similarity_top_k` 设置为 6。

In [ ]:
query_engine = get_sentence_window_query_engine(index, similarity_top_k=6)

## 三、TruLens评测

从名为 'generated_questions.text' 的文件中读取生成的问题，将其存储在 `eval_questions` 列表中。

In [ ]:
eval_questions = []
with open('data/generated_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

定义了一个函数 `run_evals`，该函数接受生成的问题列表、`TruLens` 记录器和查询引擎作为参数。对于每个问题，使用 `TruLens` 记录器开始记录，然后使用查询引擎执行查询。这个函数看起来是在评估查询结果，但是在提供的代码片段中并没有实际评估的逻辑。

In [ ]:
from trulens_eval import Tru

def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

使用 Tru 类的 `reset_database` 方法重置 `TruLens` 数据库。

In [ ]:
from utils import get_prebuilt_trulens_recorder

from trulens_eval import Tru

Tru().reset_database()

### 3.1 滑窗尺寸设置为1

调用之前定义的函数 `build_sentence_window_index` 和 `get_sentence_window_query_engine`, 分别构建了句子窗口索引和查询引擎。这里设置了窗口大小为 1，并指定了保存目录为 "sentence_index_1"。

In [ ]:
sentence_index_1 = build_sentence_window_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-zh-v1.5",
    sentence_window_size=1,
    save_dir="sentence_index_1",
)
sentence_window_engine_1 = get_sentence_window_query_engine(
    sentence_index_1
)
tru_recorder_1 = get_prebuilt_trulens_recorder(
    sentence_window_engine_1,
    app_id='sentence window engine 1'
)

调用之前定义的评估函数 `run_evals`，传入生成的问题列表、`TruLens` 记录器 `tru_recorder_1` 和构建的查询引擎 `sentence_window_engine_1`，运行评估任务。

In [ ]:
run_evals(eval_questions, tru_recorder_1, sentence_window_engine_1)

In [ ]:
tru_recorder_1 = get_prebuilt_trulens_recorder(
    sentence_window_engine_1,
    app_id='sentence window engine 1'
)

调用 `Tru` 类的 `run_dashboard` 方法，运行 `TruLens` 框架的仪表板,可视化结果。

In [ ]:
Tru().run_dashboard()

由于 API 调用有时会失败，您可能偶尔会看到空响应，因此需要重新运行评估。 因此，在较小的批次中运行评估也可以帮助您节省时间和成本，因为您只需在出现问题的批次中重新运行评估。

In [ ]:
eval_questions = []
with open('data/generated_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

### 3.2 滑窗尺寸设为3

调用之前定义的函数，构建了句子窗口索引、查询引擎和 `TruLens` 记录器。这里设置了窗口大小为 3，并指定了保存目录为 "sentence_index_3"。

In [ ]:
sentence_index_3 = build_sentence_window_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-zh-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index_3",
)
sentence_window_engine_3 = get_sentence_window_query_engine(
    sentence_index_3
)

tru_recorder_3 = get_prebuilt_trulens_recorder(
    sentence_window_engine_3,
    app_id='sentence window engine 3'
)

调用 `run_evals` 函数，传入生成的问题列表 `eval_questions`、`TruLens` 记录器 `tru_recorder_3` 和构建的查询引擎 `sentence_window_engine_3`，运行评估任务。

In [ ]:
run_evals(eval_questions, tru_recorder_3, sentence_window_engine_3)

调用 `Tru` 类的 `run_dashboard` 方法，运行 `TruLens` 框架的仪表板，可能是为了可视化评估结果。

In [ ]:
Tru().run_dashboard()